# PyMOL Styling

In [1]:
from biopandas.pdb import PandasPdb
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
from pymol import cmd

In [2]:
def find_polar_contacts(pdb_path: str, chain1: str, chain2: str, cutoff=3.5):
    """
    Find polar contacts between two input chains in a PDB file using BioPandas.
    
    Parameters:
    pdb_path (str): Path to the PDB file.
    chain1 (str): Identifier for the first chain.
    chain2 (str): Identifier for the second chain.
    cutoff (float): Distance cutoff for identifying polar contacts.
    
    Returns:
    dict: A dictionary with chain identifiers as keys and lists of residue numbers involved in polar contacts as values.
    """
    ## Load PDB file
    ppdb = PandasPdb().read_pdb(pdb_path)
    df = ppdb.df['ATOM']

    ## Select polar atoms (N, O) from each chain
    polar_atoms_chain1 = df[(df['chain_id'] == chain1) & (df['element_symbol'].isin(['N', 'O']))]
    polar_atoms_chain2 = df[(df['chain_id'] == chain2) & (df['element_symbol'].isin(['N', 'O']))]

    ## Calculate pairwise distances between polar atoms
    distances = cdist(polar_atoms_chain1[['x_coord', 'y_coord', 'z_coord']].values, 
                      polar_atoms_chain2[['x_coord', 'y_coord', 'z_coord']].values)

    ## Find pairs of atoms within the distance cutoff
    pairs = np.argwhere(distances <= cutoff)

    ## Find corresponding residue numbers and names
    residue_numbers_chain1 = polar_atoms_chain1['residue_number'].values
    residue_numbers_chain2 = polar_atoms_chain2['residue_number'].values

    residue_names_chain1 = polar_atoms_chain1['residue_name'].values
    residue_names_chain2 = polar_atoms_chain2['residue_name'].values

    polar_contact_residues = []
    for pair in pairs:
        polar_contact_residues.append([
            residue_names_chain1[pair[0]],
            residue_numbers_chain1[pair[0]],
            residue_names_chain2[pair[1]],
            residue_numbers_chain2[pair[1]]
            ])

    ## Make polar contact residues a set
    polar_contact_residues = set(map(tuple, polar_contact_residues))

    ## Assemble dictionary
    polar_contact_residues_dict = {
        f"resn_chain{chain1}": [],
        f"resi_chain{chain1}": [],
        f"resn_chain{chain2}": [],
        f"resi_chain{chain2}": []
        }
    
    for pair in polar_contact_residues:
        polar_contact_residues_dict[f"resn_chain{chain1}"].append(pair[0])
        polar_contact_residues_dict[f"resi_chain{chain1}"].append(pair[1])
        polar_contact_residues_dict[f"resn_chain{chain2}"].append(pair[2])
        polar_contact_residues_dict[f"resi_chain{chain2}"].append(pair[3])

    ## Convert to dataframe
    polar_contact_residues_df = pd.DataFrame(polar_contact_residues_dict)

    return polar_contact_residues_df

In [12]:
contacts_df = pd.read_csv('polar_contacts.csv')

## Filter to a list of experiment_ids
experiment_ids_to_show = [
    'AVFluIgG01__EPI168674', ## Worst Overall
    '100F4__EPI2429052', ## Best Overall
    '13D4__EPI3358339', ## Best Mexican isolate
    '12H5__EPI658567', ## Improved with G225R (Best)
    # 'H5.3__EPI167553', ## Worsened with N224K (Worst)
    'FLD194__EPI3178330', ## Worsened with E190N (2nd Best)
    '65C6__EPI242227' ## Oldest Isolate with N158D (Best)
    ]

contacts_df = contacts_df[contacts_df['experiment_id'].isin(experiment_ids_to_show)]

contacts_df

,experiment_id,pdb_path,pos_111,pos_112,pos_113,pos_114,pos_115,pos_116,pos_117,pos_118,...,pos_261,pos_262,pos_263,pos_264,pos_265,pos_266,pos_267,pos_268,pos_269,pos_270
54,FLD194__EPI3178330,../../../data/results/FLD194__EPI3178330/outpu...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
477,AVFluIgG01__EPI168674,../../../data/results/AVFluIgG01__EPI168674/ou...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
898,65C6__EPI242227,../../../data/results/65C6__EPI242227/output/0...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
1408,12H5__EPI658567,../../../data/results/12H5__EPI658567/output/0...,0,0,0,0,1,0,0,1,...,1,1,0,0,0,1,0,0,0,0.0
1622,100F4__EPI2429052,../../../data/results/100F4__EPI2429052/output...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
1795,13D4__EPI3358339,../../../data/results/13D4__EPI3358339/output/...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [13]:
def highlight_residues(object:str="obj01", chain:str="A", residues:list=[], style:str="sticks", color:str="red"):
    """
    Highlight residues in a chain in a given style and color.

    Parameters:
    object (str): Object name.
    chain (str): Chain identifier.
    residues (list): List of residue numbers.
    style (str): Style to use for highlighting.
    color (str): Color to use for highlighting.
    """
    ## Generate a selection string for the residues
    selection = " or ".join([f"({object} and chain {chain} and resi {res})" for res in residues])
    ## Generate a selection name
    selection_name = f"highlight_{object}_{chain}"
    ## Create the selection in PyMOL
    cmd.select(selection_name, selection)
    ## Show the selection as sticks
    cmd.show(style, selection_name)
    ## Color the selection red
    cmd.color(color, selection_name)


In [14]:
chain1 = "A"
chain2 = "B"
# chain1_color = "0x005035"
# chain2_color = "0xA49665"
chain1_color = "0x4C4C4C" ## Grey
chain2_color = "0x732466" ## PMS 255
cutoff = 3.5

cmd.reinitialize("everything")

for index, experiment in contacts_df.iterrows():
    experiment_id = experiment['experiment_id']
    print(f"Experiment ID: {experiment_id}")
    pdb_path = experiment['pdb_path']
    contacts = find_polar_contacts(pdb_path, chain1, chain2, cutoff)

    cmd.load(pdb_path, experiment_id)

    ## Color chain A
    cmd.color(chain1_color, f"{experiment_id}, chain {chain1}")

    ## color chain B
    cmd.color(chain2_color, f"{experiment_id}, chain {chain2}")

    ## Highlight Chain A
    highlight_residues(object=experiment_id,
                       chain = "A",
                       residues = list(set(contacts["resi_chainA"].tolist())),
                       color = chain1_color)

    ## Highlight Chain B
    highlight_residues(object=experiment_id,
                       chain = "B",
                       residues = list(set(contacts["resi_chainB"].tolist())),
                       color = "red")
    
    ## Set chain A opacity
    cmd.set("cartoon_transparency", 0.5, f"chain {chain1}")

## Recolor all chain A's
cmd.color(chain1_color, f"chain {chain1}")

## Load reference structure
# cmd.load("1rd8_HA1_chainA.pdb", "1rd8")

## Align everything to reference structure
# cmd.align("*", "1rd8")

## Remove reference structure
# cmd.delete("1rd8")

## Zoom to fit
cmd.zoom("all")

## Save PyMOL Session
cmd.save("subset_polar_contacts_session.pse")

# cmd.reinitialize("everything")


Experiment ID: FLD194__EPI3178330
Experiment ID: AVFluIgG01__EPI168674
Experiment ID: 65C6__EPI242227
Experiment ID: 12H5__EPI658567
Experiment ID: 100F4__EPI2429052
Experiment ID: 13D4__EPI3358339


---------------------
### Temp Viewing

In [ ]:
## Initialize temp file for rendering
import tempfile

tmp_png = tempfile.NamedTemporaryFile(delete=False, prefix=f"temp_", suffix=".png")
print(tmp_png.name)

In [ ]:
cmd.orient()
# cmd.do("ray")
cmd.png(tmp_png.name, width=1920, height=1080, ray=1)

In [ ]:
from IPython.display import Image
Image(tmp_png.name)

-----------------------------

```py
## Goodsell-Like Style
unset specular
set ray_trace_gain, 0
# set ray_trace_mode, 3
set ray_trace_mode, 4
bg_color white
set ray_trace_color, black
unset depth_cue
ray
```